In [74]:
import datasets
import pandas as pd
import os

In [75]:
# actionbility_mapping = {
#     "Unactionable": "1", 
#     "Borderline Actionable": "2", 
#     "Somewhat Actionable": "3", 
#     "Mostly Actionable": "4", 
#     "Highly Actionable": "5"
# }


def map_values(example):
    example["actionability"] = example["actionability"][0] if 'actionability' in example.keys() and example["actionability"] else None
    example["grounding_specificity"] = example["grounding_specificity"][0] if 'grounding_specificity' in example.keys() and example["grounding_specificity"] else None
    example['verifiability'] = example['verifiability'][0] if 'verifiability' in example.keys() and example['verifiability'] else None
    example['helpfulness'] = example['helpfulness'][0] if 'helpfulness' in example.keys() and example['helpfulness'] else None
    example['professional_tone'] = example['professional_tone'][0] if 'professional_tone' in example.keys() and example['professional_tone'] else None
    example['valid_point'] = example['valid_point'][0] if 'valid_point' in example.keys() and example['valid_point'] else None
    example['addressed_to_author'] = example['addressed_to_author'][0] if 'addressed_to_author' in example.keys() and example['addressed_to_author'] else None
    if example['addressed_to_author'] == 'authors':
        example['addressed_to_author'] = '1'
    elif example['addressed_to_author'] == 'meta_reviewer':
        example['addressed_to_author'] = '0'
    else :
        example['addressed_to_author'] = None
    return example

In [76]:
BATCH = 8

dataset_folder = f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_{BATCH}'
human_annotations = []
## get all subdirectories in the dataset folder


with pd.ExcelWriter(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_{BATCH}/human_annotation_{BATCH}.xlsx', engine="openpyxl") as writer:

    for human_data in os.listdir(dataset_folder):
        
        dataset_name = os.path.join(dataset_folder, human_data)

        ## if this is not a directory, skip
        if not os.path.isdir(dataset_name):
            continue

        ds = datasets.load_from_disk(dataset_name)
  
        column_names  = {
            'actionability.responses' : 'actionability', 
            'grounding_specificity.responses' : 'grounding_specificity',
            'verifiability.responses' : 'verifiability',
            'helpfulness.responses' : 'helpfulness',
            'professional_tone.responses' : 'professional_tone',
            'valid_point.responses' : 'valid_point',
            'addressed_to_author.responses' : 'addressed_to_author',}
        ## delete columns that are not in the dataset
        column_names = {key: value for key, value in column_names.items() if key in ds.column_names}
        ds = ds.rename_columns(column_names)
        
        selected_columns = ['id','review_point', 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness', 'professional_tone', 'valid_point','addressed_to_author']
        ## select columns of they exist
        selected_columns = [column for column in selected_columns if column in ds.column_names]
        ds = ds.select_columns(selected_columns)

        ds = ds.map(map_values)

        df = ds.to_pandas()


        df.to_excel(writer, sheet_name=human_data, index=False)


In [77]:
df

,id,review_point,actionability,grounding_specificity,verifiability,helpfulness,professional_tone,valid_point,addressed_to_author
0,train_0,- There were too many missing details (for exa...,Mostly Actionable,5,2,3,0,1,None
1,train_1,"- Section 4.3, model parameters and training. ...",Borderline Actionable,3,3,2,0,0,None
2,train_2,- The differences in results in Table 2 are ve...,Unactionable,1,2,2,0,1,None
3,train_3,"- While the language has been improved, there ...",Unactionable,5,X,2,0,0,None
4,train_4,1) The character tri-gram LSTM seems a little ...,Highly Actionable,5,5,5,1,0,None
5,train_5,- It is difficult to determine the impact of s...,Highly Actionable,4,1,3,0,0,None
6,train_6,2. A minor point would be that few-shot would ...,Highly Actionable,5,4,4,1,0,None
7,train_7,- From Figure 3 it is not clear to me how ZX i...,Highly Actionable,4,5,5,1,0,None
8,train_8,5. The authors may choose to insert into Figur...,Unactionable,1,1,2,0,0,None
9,train_9,- Improvements on Multi30k are quite small (< ...,Highly Actionable,5,5,5,1,0,None


In [78]:
for annotation in human_annotations:
    annotator